# This is a Tensorflow Training Notebook

Below is online courses project code of IBM Deep Learning with Tensorflow

In [3]:
import tensorflow as tf
from keras.api.layers import Layer
from keras.api.models import Sequential
# from tensorflow.python.keras.models import Sequential as tf_sequential
# tf_sequential([Layer(), Layer(), Layer()])
# something different with keras.api.models.Sequential and tensorflow.api.keras.models.Sequential
# Cuz when I use tensorflow.api.keras.models.Sequential([Layer1, Layer2, Layer3]), it will raise an error
# ValueError: The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.
# And I don't know how to fix it, so I use keras.api.models.Sequential instead
# And I don't know the difference between them, so I just use keras.api.models.Sequential
# I think I should read the document and the source code to find the difference between them
# this is the constructor of keras.api.models.Sequential
# it calls the _maybe_rebuild() method
# when we call the constructor of Sequential([Laysers]), it will build the model automatically
# def __init__(self, layers=None, trainable=True, name=None):
#        super().__init__(trainable=trainable, name=name)
#        self._functional = None
#        self._layers = []
#        if layers:
#            for layer in layers:
#                self.add(layer, rebuild=False)
#            self._maybe_rebuild()
# -------------------------------------------------------------
# this is the constructor of tensorflow.python.keras.models.Sequential
# @trackable.no_automatic_dependency_tracking
#   def __init__(self, layers=None, name=None):
#     """Creates a `Sequential` model instance.
#
#     Args:
#       layers: Optional list of layers to add to the model.
#       name: Optional name for the model.
#     """
#     # Skip the init in FunctionalModel since model doesn't have input/output yet
#     super(functional.Functional, self).__init__(  # pylint: disable=bad-super-call
#         name=name, autocast=False)
#     self.supports_masking = True
#     self._compute_output_and_mask_jointly = True
#     self._auto_track_sub_layers = False
#     self._inferred_input_shape = None
#     self._has_explicit_input_shape = False
#     self._input_dtype = None
#     self._layer_call_argspecs = {}
#     self._created_nodes = set()
#     # Flag that indicate whether the sequential network topology has been
#     # created. It is false when there isn't any layer, or the layers doesn't
#     # have input shape.
#     self._graph_initialized = False
#
#     # Unfortunately some Sequential models using custom layers or FeatureColumn
#     # layers have multiple inputs. This is fundamentally incompatible with
#     # most of the Sequential API, and we have to disable a number of features
#     # for such models.
#     self._use_legacy_deferred_behavior = False
#
#     # Add to the model any layers passed to the constructor.
#     if layers:
#       if not isinstance(layers, (list, tuple)):
#         layers = [layers]
#       for layer in layers:
#         self.add(layer)
# but when we call the constructor of tensorflow.python.keras.models.Sequential([Layers]), it will not build the model automatically
# so it will raise an error and we need to build the model mannually
# **SORRY, I FORGOT THE ERROR MESSAGE, I WILL ADD IT LATER**
# -------------------------------------------------------------
# and the environment is Python 3.12.8, tensorflow 2.19.0, keras 3.9.0
# the environment of the lab exercise is Python IDK, tensorflow 2.16.2, keras IDK
# -------------------------------------------------------------
# Commit fe87e2b
# From the commit history of keras, I found that the keras has changed since Apr 14, 2023 by fchollet 
# and the commit message is Sequential improvements -- move to "always build"
# and the release version is 3.0.0, Nov 28, 2023, so I think this is a great change
# -     def pop(self):
# +     def pop(self, rebuild=True):
#          layer = self._layers.pop()
# -        self.built = False
# _        self._functional = None
# +        if rebuild:
# +            self._maybe_rebuild()
# +        else:
# +            self.built = False
# +            self._functional = None
#
## very useful change
# -------------------------------------------------------------
# and tensorflow hasn't changed this part of the code, so I think the lab exercise is based on keras api

**Step 2: Define a custom layer**

Define a custom dense layer with 32 units and ReLU activation.

In [4]:
class CustomDenseLayer(Layer):
    def __init__(self, units = 32):
        super(CustomDenseLayer, self).__init__()
        self.units = units
    
    def build(self, input_shape):
        self.w = self.add_weight(shape = (input_shape[-1], self.units),
                                 initializer = 'random_normal',
                                 trainable = True)
        self.b = self.add_weight(shape = (self.units,),
                                 initializer = 'zeros',
                                 trainable = True)
        
    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b)

Step 3: Integrate the custom layer into a model

Create a Keras model using the custom layer.

In [5]:
from keras.api.layers import Softmax

# Define the model with Softmax in the output layer

model = Sequential([
    CustomDenseLayer(128),
    CustomDenseLayer(10),   # Hidden layer with ReLU activation
    Softmax()               # Output layer with Softmax activation
])

The Softmax activation function is used in the output layer for multi-class classification tasks, ensuring the model outputs probabilities that sum up to 1 for each class, which aligns with categorical cross-entropy as the loss function. This adjustment ensures the model is optimized correctly for multi-class classification.

Step 4: Compile the model

Compile the model with the Adam optimizer and categorical cross-entropy loss.

In [6]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy')
print("Model summary before building:")
model.summary()

# Build the model to show parameters
model.build((1000, 20))
print("\nModel summary after building:")
model.summary()

Step 5: Train the model

Train the model on some example data. For this example, you will generate random data for training. In practice, use a real data set.

In [7]:
import numpy as np

# Generate random data
x_train = np.random.random((1000, 20))
y_train = np.random.randint(10, size = (1000, 1))

# Convert labels to categorical one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes = 10)
model.fit(x_train, y_train, epochs = 10, batch_size = 32)

Step 6: Evaluate the model

Evaluate the model using test data to see its performance.

For this example, you will generate random test data. In practice, use a real data set.

In [8]:
# Generate random test data
x_test = np.random.random((200, 20))
y_test = np.random.randint(10, size = (200, 1))

# Convert labels to categorical one-hot encoding
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 10)

# Evaluate the model
loss = model.evaluate(x_test, y_test)
print(f'Test loss: {loss}')

#### Exercise 1: Visualize Model Architecture

**Objective:** Visualize the architecture of the custom Keras model to understand its structure.

**Instructions:**
1. Use the `plot_model` function from `tensorflow.keras.utils` to visualize the model architecture.
2. Save the plot as an image file.

In [9]:
from keras.api.utils import plot_model
plot_model(model, to_file = 'model_architecture.png', show_shapes = True, show_layer_names = True)

Exercise 2: Add Dropout Layer
Objective: Enhance the model by adding a Dropout layer to prevent overfitting.

Instructions:

Add a Dropout layer between the custom dense layers.
Recompile the model and observe the impact on training.

In [10]:
from keras.api.layers import Dropout

# Modify the model to include a Dropout layer
model = Sequential([
    CustomDenseLayer(64),
    Dropout(0.5),           
    CustomDenseLayer(10)
])

# Recompile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Train the model again
model.fit(x_train, y_train, epochs=10, batch_size=32)

Exercise 3: Adjust the Number of Units in Custom Layer
Objective: Experiment with different numbers of units in the custom dense layer to observe the impact on performance.

Instructions:

Change the number of units in the CustomDenseLayer to 128.
Recompile, train, and evaluate the model.

In [11]:
class CustomDenseLayer2(Layer):
    def __init__(self, units=128):
        super(CustomDenseLayer2, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros',
                                 trainable=True)

    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b)

# Integrate the new custom layer into a model
model3 = Sequential([
    CustomDenseLayer2(128),
    CustomDenseLayer2(10)
])

# Recompile the model
model3.compile(optimizer='adam', loss='categorical_crossentropy')

# Train the model again
model3.fit(x_train, y_train, epochs=10, batch_size=32)